## 1. Process nb errors in Kaggle

In [3]:
import pandas as pd
import util
import config

path_err = config.path_kaggle_error_process + '/nberror_k.xlsx'

In [ ]:
# Process the traceback column to readable texts

In [11]:
df_err = pd.read_excel(path_err)

df_err['traceback'] = df_err['traceback'].map(util.parse_traceback)
df_err.to_excel(config.path_kaggle_error_process+'/nberror_k_p.xlsx', index=False, engine='xlsxwriter')

In [ ]:
### Extract meaningful and short evalues from traceback for those with ignored evalues

In [3]:
df_err = pd.read_excel(config.path_kaggle_error_process+'/nberror_k_p.xlsx')
df_err_ignored = df_err[df_err['evalue']=='ignored']
df_err_ignored.count()

fname        175
ename        175
evalue       175
traceback    175
dtype: int64

In [4]:
for index, row in df_err_ignored.iterrows():
    res = util.get_evalue_ignored_from_traceback(row)
    if res:
        df_err.loc[index, "evalue"] = res
df_err[df_err['evalue']=='ignored'].count()

exception when listing traceback
exception when listing traceback


fname        39
ename        39
evalue       39
traceback    39
dtype: int64

In [5]:
df_err.to_excel(config.path_kaggle_error_process+'/nberror_k_p2.xlsx', index=False, engine='xlsxwriter')

In [ ]:
### Simply parse libraries from crash line from traceback

In [4]:
df_err = pd.read_excel(config.path_kaggle_error_process+'/nberror_k_p2.xlsx')

In [5]:
df_err['lib'] = df_err['traceback'].apply(util.extract_lib)
print("Extract {0:2.2f}% libraries from crash traceback values".format(
    (len(df_err)-len(df_err[df_err['lib'].isnull()]))/len(df_err)))
df_err.to_excel(config.path_kaggle_error_process+'/nberror_k_p3.xlsx', index=False, engine='xlsxwriter')

Extract 0.52% libraries from crash traceback values


In [ ]:
# again

import pandas as pd
import pickle
import util

path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error"
df_err = pd.read_excel(path_err_nbs + '/nberror_k_p3.xlsx')
df_imports = pd.read_excel(path_err_nbs + '/imports_all_info.xlsx')

df_err["lib2"] = df_err.apply(util.extract_lib_2, lib_names=config.top_lib_names, df_imports=df_imports, lib_classes_dict=lib_classes_dict, axis=1)

1-len(df_err[df_err.lib.isnull()&df_err.lib2.isnull()])/len(df_err)

In [ ]:
df_err["lib_parsed"] = df_err["lib2"].fillna(df_err["lib"]).map(util.simple_lib_parser)
df_err.to_excel(path_err_nbs+"/nberror_k_all_p3_lib_parsed.xlsx", index=False, engine='xlsxwriter')

## 2. Process nb errors in Github

### 3.1 Process the traceback column to readable texts

In [10]:
import pandas as pd
import util
import config

df2_err = pd.read_excel(config.path_github_error_process + '/nberror_g_all.xlsx')

df2_err['traceback'] = df2_err['traceback'].map(util.parse_traceback)
df2_err.to_excel(config.path_github_error_process + '/nberror_g_all_p.xlsx', index=False, engine='xlsxwriter')

### 3.2 Extract meaningful and short evalues from traceback for those with ignored evalues

Example:

    fname                             00000-1828-homework1-1.ipynb
    ename                                                NameError
    evalue                                                 ignored
    traceback    ['---------------------------------------------------------------------------', 'NameError                                 Traceback (most recent call last)', '<ipython-input-8-287bf352fb78> in <module>()\n      3 c1= nd.norm(nd.dot(A, B))\n      4 c= mx.ndarray.power(c1,2)\n----> 5 tic= time.time()\n      6 print(time.time() - tic)\n      7 \n', "NameError: name 'time' is not defined"]


In [12]:
import pandas as pd
import util
import config

df2_err = pd.read_excel(config.path_github_error_process + '/nberror_g_all_p.xlsx')
df2_err_ignored = df2_err[df2_err['evalue']=='ignored']
df2_err_ignored.count()

fname        17478
ename        17466
evalue       17478
traceback    17478
dtype: int64

In [92]:
for index, row in df2_err_ignored.iterrows():
    res = util.get_evalue_ignored_from_traceback(row)
    if res:
        df2_err.loc[index, "evalue"] = res

In [93]:
df2_err[df2_err['evalue']=='ignored'].count() # managed to filled 12800/17478 ignored evalues

fname        4678
ename        4666
evalue       4678
traceback    4678
dtype: int64

In [96]:
df2_err.to_excel(config.path_github_error_process + '/nberror_g_all_p2.xlsx', index=False, engine='xlsxwriter')

### 3.4 Parse libraries from traceback

In [1]:
import pandas as pd
import util
import config

path = config.path_github_error_process + '/nberror_g_all_p2.xlsx'
df2_err = pd.read_excel(path)

In [5]:
df2_err['lib'] = df2_err['traceback'].apply(util.extract_lib)

In [23]:
print("Extract {0:2.2f}% libraries from crash traceback values".format(
    (len(df2_err)-len(df2_err[df2_err['lib'].isnull()]))/len(df2_err)))

Extract 0.35% libraries from crash traceback values


#### rules for extracting related libs from error information:

~~(not included) 1. Uninteresting error types even without knowing if any libraries are involved:
        NameError?, FileNotFoundError, OSError, KeyboardInterrupt?, ModuleNotFoundError?, AssertionError?~~
        
    2. compile a list of popular libraries used in data science and ML
        1. if library name is in error value. e.g., 
            TypeError: 'numpy.float64' object is not callable
            AttributeError: module 'torchvision.transforms.v2' has no attribute 'CutMix'
        2. for pattern "----> line_number xx.yy", check if xx is alias of an import library
        ??? 3. similarly, for yy, can check if the property name is from a library. Do not know how.
        4. for some type errors and attribute errors, where to check the object name is from which library.
            AttributeError: 'Sequential' object has no attribute 'predict_classes'
            TypeError: 'AxesSubplot' object is not subscriptable
        ??? 5. for ValueError, certain keywords in error values indicates library names
            broadcast, array, shape -> numpy
            

In [ ]:
# lib2 parsing config
import pickle
import config

#config.top_lib_names
with open('lib_classes.pickle', 'rb') as f:
    lib_classes_dict = pickle.load(f)

In [ ]:
path_err_nbs = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error"
df_imports = pd.read_excel(path_err_nbs + '/imports_all_info.xlsx')
df2_err["lib2"] = df2_err.apply(util.extract_lib_2, lib_names=config.top_lib_names, df_imports=df_imports, lib_classes_dict=lib_classes_dict, axis=1)

In [ ]:
df2_err["lib_parsed"] = df2_err["lib2"].fillna(df2_err["lib"]).map(util.simple_lib_parser)
df2_err.to_excel(path_err_nbs+"/nberror_g_all_p4_exception_types_lib_parsed.xlsx", index=False, engine='xlsxwriter')
sum(~df2_err["lib_parsed"].isna())/len(df2_err)

In [25]:
df2_err.to_excel(config.path_github_error_process + '/nberror_g_all_p4.xlsx', index=False, engine='xlsxwriter')